In [1]:
from aocd import data, submit
from util import *
from typing import List
from collections import defaultdict

In [2]:
data.splitlines()[:5]

['Tile 2903:', '..#..#....', '.....#...#', '..#...#.##', '#....#..#.']

In [3]:
sample_data = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###..."""

In [4]:
from itertools import takewhile
import numpy as np

def parse_data(data):
    data_iter = iter(data.splitlines())
    
    raw_tile_groups = {}
    while True:
        next_group = list(takewhile(
            lambda x: x != "",
            data_iter
        ))
        if next_group == []:
            break
        tile_id = int(next_group[0][4:-1])
        raw_tile_groups[tile_id] = np.array(list(map(list, next_group[1:])))
    
    return raw_tile_groups
        

In [5]:
# collect the 8 mirrored versions of the sides

In [37]:
def get_all_slides(group) -> List[str]:
    # collect the 4 sides
    sides = [
        group[0, :],
        group[:, -1],
        group[-1, :],
        group[:, 0]
    ]
    
    # Add mirrors
    sides.extend(mapl(lambda x: x[::-1], sides))
    
    return mapl(lambda x: "".join(x), sides)
    

In [38]:
def build_side_id_map(groups):
    side_id_map = defaultdict(set)
    for tile_id, group in groups.items():
        for side in get_all_slides(group):
            side_id_map[side].add(tile_id)
    return side_id_map

In [8]:
# find out which tiles have more than 2 solo sides

In [9]:
from collections import Counter

In [10]:
def find_corner_ids_product(side_id_map):
    single_side_counter = Counter()
    for tile_set in side_id_map.values():
        if len(tile_set) == 1:
            single_side_counter[list(tile_set)[0]] += 1
    corners = filterl(lambda x: x[1] > 2, single_side_counter.items())
    assert len(corners) == 4
    corner_ids = mapl(lambda x: x[0], corners)
    return corner_ids

In [220]:
groups = parse_data(data)
side_id_map = build_side_id_map(groups)
result = np.product(find_corner_ids_product(side_id_map))

In [221]:
result

19955159604613

In [222]:
# submit(result)

# Part 2

In [223]:
def rotate_right(array, times = 1):
    return np.rot90(array, k = - times)

def align_tile_top(tile: np.array, side_index: int):
    # index from get_all_slides()
    if side_index == 0:
        return tile
    if side_index == 1:
        return rotate_right(tile, 3)
    if side_index == 2:
        return np.fliplr(rotate_right(tile, 2))
    if side_index == 3:
        return np.fliplr(rotate_right(tile, 1))
    if side_index == 4:
        return np.fliplr(tile)
    if side_index == 5:
        return np.fliplr(rotate_right(tile, 3))
    if side_index == 6:
        return rotate_right(tile, 2)
    if side_index == 7:
        return rotate_right(next_tile, 1)

In [224]:
import math

In [225]:
SIZE = int(math.sqrt(len(groups)))

In [250]:
from copy import copy 
# find actual complete ordering of tiles
assembled_tiles = []
assembled_tile_ids = set()

# index scheme:
# 0 2
# 1 3

# start with 1 of the corners to anchor things
# find corner to start and orient it so the outsides are up/left
tile_id = find_corner_ids_product(side_id_map)[0]
tile = groups[tile_id]

sides = get_all_slides(tile)[:4] # only need first four since we won't need to flip
out_sides = mapl(lambda side: len(side_id_map[side]) == 1, sides)
while out_sides != [True, False, False, True]:
    tile = rotate_right(tile)
    sides = get_all_slides(tile)[:4] # only need first four since we won't need to flip
    out_sides = mapl(lambda side: len(side_id_map[side]) == 1, sides)

assembled_tiles.append(tile)
assembled_tile_ids.add(tile_id)

while len(assembled_tiles) < SIZE * SIZE:
    # print(f"Starting {len(assembled_tiles)}")
    if len(assembled_tiles) % SIZE == 0:
        # start a new column off
        # target right side of top of prevoius column
        target_side = "".join(assembled_tiles[-1 * SIZE][:, -1])
        # find other tile with this size, should only be 1
        matching_tiles = side_id_map[target_side]
        next_tile_id = list(matching_tiles - assembled_tile_ids)[0]
        next_tile = groups[next_tile_id]
        next_sides = get_all_slides(next_tile)
        side_index = next_sides.index(target_side)
        top_aligned_tile = align_tile_top(next_tile, side_index)
        # instead of being top aligned, just gotta rotate this guy slightly
        final_tile = rotate_right(np.fliplr(top_aligned_tile), 3)
    else:
        # find the next value down the list
        # start with the bottom of side of the previous tile
        target_side = "".join(assembled_tiles[-1][-1, :])
        # find other tile with this size, should only be 1
        matching_tiles = side_id_map[target_side] - assembled_tile_ids
        if len(matching_tiles) != 1:
            import pdb; pdb.set_trace()
        next_tile_id = list(matching_tiles)[0]
        next_tile = groups[next_tile_id]
        next_sides = get_all_slides(next_tile)
        side_index = next_sides.index(target_side)
        final_tile = align_tile_top(next_tile, side_index)

    assembled_tiles.append(final_tile)
    assembled_tile_ids.add(next_tile_id)
    
assert assembled_tile_ids == set(groups.keys())

In [251]:
blocked_arrays = []
full_blocked_arrays = [] # for debugging
for i in range(SIZE):
    current = []
    full_current = []
    for j in range(SIZE):
        # drop the outside of each block to get at the actual image
        current.append(assembled_tiles[j * SIZE + i][1:-1, 1:-1])
        full_current.append(assembled_tiles[j * SIZE + i])
    
    blocked_arrays.append(current)
    full_blocked_arrays.append(full_current)

In [252]:
combined = np.block(blocked_arrays) # this is the final, reconstructed image
full_combined = np.block(full_blocked_arrays)

In [ ]:
# If desired, print out the full thing for debugging
for row in full_combined:
    print("".join(row[:11]))

In [ ]:
# Print final image
for row in combined:
    print("".join(row))

### Now find monsters

In [234]:
monster_str = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
""".replace(" ", "0").replace("#", "1").splitlines()

In [235]:
monster_mask = np.array(mapl(lambda l: mapl(int, l), mapl(list, monster_str)))
MONSTER_SIZE = np.sum(monster_mask)

In [236]:
monster_mask

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]])

In [237]:
 import numpy.ma as ma

In [269]:
def test_monster(i, j, array):
    full_mask = np.zeros(array.shape, dtype=np.bool)
    full_mask[i:monster_mask.shape[0]+i, j:monster_mask.shape[1] + j] = monster_mask
    return np.sum(array[full_mask] == '#') == MONSTER_SIZE, full_mask

In [276]:
# Try all 8 rotation/flip combos
candidates = [
    combined,
    rotate_right(combined, 1),
    rotate_right(combined, 2),
    rotate_right(combined, 3),
    np.fliplr(combined),
    rotate_right(np.fliplr(combined), 1),
    rotate_right(np.fliplr(combined), 2),
    rotate_right(np.fliplr(combined), 3),
]
for candidate in candidates:
    monster_free = candidate.copy()
    found = False
    for i in range(candidate.shape[0] - monster_mask.shape[0]):
        for j in range(candidate.shape[1] - monster_mask.shape[1]):
            result, mask = test_monster(i, j, candidate)
            if result:
                found = True
                # print(f"Found one at {i}, {j}")
                # mask out the monster indices so they won't count at the end
                monster_free = np.ma.array(monster_free, mask=mask).filled('0')
    if found:
        break

In [ ]:
# print out monster masked image
for row in monster_free:
    print("".join(row))

In [277]:
result = np.sum(monster_free == '#')

In [278]:
result

1639

In [274]:
submit(result)

answer a: 19955159604613
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 20! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>